In [ ]:
import pandas as pd
import numpy as np

def calculate_supertrend(df, period=10, multiplier=3):
    # Calculate ATR
    high_low = df['high'] - df['low']
    high_prev_close = abs(df['high'] - df['close'].shift())
    low_prev_close = abs(df['low'] - df['close'].shift())
    tr = pd.DataFrame({'hl': high_low, 'hpc': high_prev_close, 'lpc': low_prev_close}).max(axis=1)
    atr = tr.ewm(span=period, adjust=False).mean()

    # Calculate basic upper and lower bands
    basic_upper_band = ((df['high'] + df['low']) / 2) + (multiplier * atr)
    basic_lower_band = ((df['high'] + df['low']) / 2) - (multiplier * atr)

    # Calculate final upper and lower bands
    final_upper_band = np.zeros(len(df))
    final_lower_band = np.zeros(len(df))
    supertrend = np.zeros(len(df))
    trend = np.zeros(len(df)) # 1 for uptrend, -1 for downtrend

    for i in range(1, len(df)):
        if df['close'].iloc[i] > final_upper_band[i-1]:
            trend[i] = 1 # Uptrend
        elif df['close'].iloc[i] < final_lower_band[i-1]:
            trend[i] = -1 # Downtrend
        else:
            trend[i] = trend[i-1] # No change

        if trend[i] == 1:
            final_lower_band[i] = max(basic_lower_band.iloc[i], final_lower_band[i-1] if trend[i-1] == 1 else basic_lower_band.iloc[i])
            final_upper_band[i] = basic_upper_band.iloc[i]
            supertrend[i] = final_lower_band[i]
        else: # Downtrend
            final_upper_band[i] = min(basic_upper_band.iloc[i], final_upper_band[i-1] if trend[i-1] == -1 else basic_upper_band.iloc[i])
            final_lower_band[i] = basic_lower_band.iloc[i]
            supertrend[i] = final_upper_band[i]

    df['supertrend'] = supertrend
    return df

In [ ]:
from lightweight_charts import Chart
import pandas as pd

# Assuming df contains your OHLCV data and 'supertrend' column from the previous step
# df = calculate_supertrend(df) 

chart = Chart()
chart.set(df) # Set your OHLCV data

# Add the Supertrend line series
chart.create_line_series(
    data=[{'time': row['time'], 'value': row['supertrend']} for index, row in df.iterrows()],
    line_width=2,
    color='blue', # Or change color based on trend
    title='Supertrend'
)

chart.show(block=True)